In [1]:
from manim import *

from rc_lib import math_types as T
from rc_lib import style
from rc_lib.math_utils import geometry
from rc_lib.design_utils import plate, sketch
from rc_lib.view_utils import title_sequence

quality = "ql"

Manim Community v0.17.2

In [2]:
inner_color = style.Pallette.GREEN
boundary_color = style.Pallette.BLUE
factory = plate.PlateCircleFactory()
factory.set_inner_color(inner_color).set_outer_color(boundary_color)

title = title_sequence.TitleSequence(default_color=boundary_color)

# TODO:
# add underscores
# plategroup factory
# basic website
# extract animations (seperation of concerns pls, make generic where it makes sense)
# parameterize access (key dict?)
# trim
# delete time enum, add to style guide
# update style guide
# sketch line copy

The reccomended way to create plates is by first drawing your holes and other relevant geometry, marking out the outer boundary using a second set of circles, and then connecting the boundary with tangent lines.

In [3]:
%%manim -v WARNING --disable_caching -$quality IntakePlateScene

class IntakePlateScene(Scene):
    def setup(self):
        small_base = factory.make_generator(0.15, 0.2)
        medium_base = factory.make_generator(0.4, 0.2)

        front_hole = geometry.point_2d(-4, -3)
        middle_hole = geometry.point_2d(-1.5, 0.25)
        back_hole = geometry.point_2d(2.5, 1.5)
        back_offset = geometry.point_2d(0.8, 0.75)

        points = [
            medium_base(front_hole),
            medium_base(middle_hole),
            medium_base(back_hole),
            small_base(back_hole + back_offset),
            small_base(back_hole + geometry.point_2d(1, -0.2)),
            small_base((middle_hole + back_hole) / 2),
            small_base((front_hole + middle_hole) / 2)
        ]
        boundary_order = [1, 3, 4, 0]
        self.plate_group = plate.PlateGroup(points, boundary_order, boundary_color=boundary_color)
        title.reset()

    def construct(self):
        self.play(title.next("Draw plate holes", color=inner_color))
        self.play(self.plate_group.draw_inner_circles(lag_ratio=0.5))
        self.wait()

        self.play(title.next("Add larger circles"))
        self.play(self.plate_group.draw_outer_circles(lag_ratio=0.5))
        self.wait()

        self.play(title.next("Connect boundary"))
        self.play(self.plate_group.draw_boundary(lag_ratio=0.75, run_time=5))
        # self.wait()

        # self.play(title.next("Trim", color=boundary_color))
        # self.play(plate_group.trim(), run_time=style.Time.SLOW * 3)

        self.wait(style.END_DELAY)

In the event a hole is too close to the edge of a plate, you may need to redraw the boundary.

In [4]:
%%manim -v WARNING --disable_caching -$quality BoundaryRedrawScene

class BoundaryRedrawScene(Scene):
    def setup(self):
        generator = factory.make_generator(1.75, 0.75)
        self.left = generator(geometry.point_2d(-6, -2))
        self.right = generator(geometry.point_2d(6, -2))
        self.middle = factory.make(1, 0.75, geometry.point_2d(0, -0.75))

        self.line = plate.PlateCircle.tangent_line(self.left, self.right, style.Palette.Color)
        self.add(self.left, self.right, self.line, self.middle.inner_circle())

        title.reset()

    def construct(self):
        self.play(title.next("Add outer circle"))
        self.play(self.middle.draw_outer_circle())

        self.play(title.next("Redraw boundary"))
        self.play(Uncreate(self.line))
        self.wait(0.25)
        self.play(Create(plate.PlateCircle.tangent_line(self.left, self.middle, boundary_color)))
        self.play(Create(plate.PlateCircle.tangent_line(self.middle, self.right, boundary_color)))

        self.wait(style.END_DELAY)

AttributeError: 'PlateCircleFactory' object has no attribute 'get_generator'

In [ ]:
%%manim -v WARNING -$quality LineConstraintScene
import operator
from rc_lib.design_utils.sketch import LineEnd

class LineConstraintScene(Scene):
    def setup(self):
        generator = factory.make_generator(1.75, 0.75)
        self._left = generator(geometry.point_2d(-6, -2))
        self._right = generator(geometry.point_2d(6, -2))
        self.add(self._left, self._right)

        self._tangent_points = plate.PlateCircle.tangent_points(self._left, self._right)

        left_start_point = self._tangent_points[0] + geometry.point_2d(1.75, 0.75)
        right_start_point = self._tangent_points[1] + geometry.point_2d(-2, 0.5)
        self._line = sketch.SketchLine(left_start_point, right_start_point, color=boundary_color)

        title.reset()
    
    def get_vars(self, *keys: str, line_end: LineEnd):
        return [_get_var(key) for key in keys]

    def _get_var(self, key: str, line_end: LineEnd):
        attributes = {
            'circle' : ('_left', '_right'),
            'dot' : ('_line.get_start', '_line.get_end'),
            'point' : ('_line.start_point', '_line.end_point'),
            'move_function' : ('_line.move_to_start', '_line.move_to_end'),
            'tangent_point' : ('_tangent_points') # tangent_points is already indexable
        }
        return operator.attrgetter(attributes[key][0 if left else 1])(self)

    def construct(self):
        self.play(title.next("Create line"))
        self.play(self._line.create())

        self.play(title.next("Add coincident constraints"))
        self.do_coincident_move(LineEnd.START)
        self.do_coincident_move(False)

        self.play(title.next("Add tangent constraints"))
        self.do_tangent_move(True)
        self.do_tangent_move(False)

        self.wait(style.END_DELAY)
    
    def do_coincident_move(self, line_end: LineEnd):
        self._do_flash(line_end)
        move_function = get_vars('move_function', line_end)
        if left:
            move_function = self._line.move_start
        else:
            move_function = self._line.move_end
        self.play(move_function(self._coincident_point(left)))

    def _coincident_point(self, line_end: LineEnd) -> T.Point2d:
        if left:
            point = self._line.start_point()
            circle = self._left
        else:
            point = self._line.end_point()
            circle = self._right
        return circle.center() + geometry.normalize(point - circle.center()) * circle.outer_radius()
    
    def do_tangent_move(self, line_end: LineEnd):
        self._do_flash(line_end)
        move_function, point, tangent_point, circle = self.get_vars('circle', 'tangent_point', 'point', 'move_function', line_end)

        angle = self._tangent_angle(line_end)
        self.play(move_function(tangent_point, path_arc=angle, path_arg_centers=[circle.center()]))

    def _tangent_angle(self, line_end: LineEnd, point: T.Point2d, tangent_point: T.Point2d, center: T.Point2d) -> float:
        circle, tangent_point, point = self.get_vars('point', 'tangent_point', 'circle')
        return (1 if line_end == LineEnd.START else -1) * angle_between_vectors(point - circle.center(), tangent_point - circle.center())
    
    def _do_flash(self, line_end: LineEnd):
        dot, circle = self.get_vars('circle', 'dot', line_end)
        # self.play(Flash(dot, run_time=0.75))
        self.play(Flash(circle, flash_radius=circle.outer_radius(), num_lines=40, run_time=0.75))

AttributeError: module 'rc_lib.style' has no attribute 'Time'